<a href="https://colab.research.google.com/github/RohithJ11/NLP_Privacy_Policies/blob/main/Text_to_Text_Transfer_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### installing TensorFlow and the Transformers libraries

In [ ]:
!pip install tensorflow transformers


### Importing nltk library (Natural Language Toolkit)

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Defining and Applying the Keyword Extraction Function

In [ ]:
import nltk
from nltk.probability import FreqDist
nltk.download('punkt')

# Function to extract keywords from text based on frequency
def extract_keywords(text, top_n=5):
    # Tokenize the words
    words = nltk.word_tokenize(text)
    # Calculate frequency distribution
    freq_dist = FreqDist(words)
    # Extract the top n keywords
    keywords = [word for word, freq in freq_dist.most_common(top_n)]
    return keywords

# Ensure this line is after the 'nltk' and function definitions
df['Keywords'] = df['Summary'].apply(extract_keywords)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### loading dataset by using the T5 model to generate summaries for the text in dataset, and applying a simple frequency-based method to extract keywords

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import tensorflow as tf

# Load the dataset
df = pd.read_csv('/content/Privacyplcy_DS_3.csv')  # Adjust the path as necessary

# Initialize the model and tokenizer
model_name = 't5-small'  # You can choose other versions like 't5-base' or 't5-large' depending on your resource availability
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Function to summarize text
def summarize_text(text):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Apply summarization to each row in the DataFrame
df['Summary'] = df['Summary_of_Content'].apply(summarize_text)  # Assuming the text column is named 'PolicyText'

# Function to extract keywords from text based on frequency
def extract_keywords(text, top_n=5):
    words = text.split()
    freq_dist = nltk.FreqDist(words)
    keywords = [word for word, freq in freq_dist.most_common(top_n)]
    return keywords

# Apply keyword extraction on the summaries
df['Keywords'] = df['Summary'].apply(extract_keywords)

# Display the DataFrame to verify the results
#print(df[['Summary', 'Keywords']])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


                                              Summary  \
0   Directly collected data includes personal info...   
1   Pfizer's use of personal data is detailed acro...   
2   this section elucidates how Pfizer may share p...   
3   users have the right to request access to, cor...   
4   Pfizer and its partners use cookies, tracking ...   
5   Pfizer employs technical, administrative, and ...   
6   users have the ability to update their persona...   
7   Pfizer will retain users' personal data for as...   
8   the section outlines the scope and limitations...   
9   the types of information collected and receive...   
10  Slack processes Customer Data according to a c...   
11  customers, administrators, and authorized user...   
12  Slack retains Customer Data based on the custo...   
13  Slack holds international recognized security ...   
14  individuals in the european economic area, the...   
15  Epson collects personal information from users...   
16  the section also directs us

Saving the output with summaries and keywords into a CSV file which cab be downloaded

In [ ]:
# Save the DataFrame with summaries and keywords to a new CSV file
output_filename = '/content/privacy_policy_summaries_keywords.csv'
df.to_csv(output_filename, index=False)

# Inform the user about the saved file
print(f'Saved the output to {output_filename}')


Saved the output to /content/privacy_policy_summaries_keywords.csv
